# Working with Symbolic Expressions

## 0.1 Finding an exact derivative with a computer algebra system

## 0.2 Doing symbolic algebra in Python

In [89]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

# 1 Modeling algebraic expressions

## 1.1 Breaking an expression into pieces

## 1.2 Building an expression tree

## 1.3 Translating the expression tree to Python

In [90]:
class Power():
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

In [91]:
class Number():
    def __init__(self,number):
        self.number = number

class Variable():
    def __init__(self,symbol):
        self.symbol = symbol

This represents $x^2$:

In [92]:
Power(Variable("x"),Number(2))

In [93]:
class Product():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

This represents $3x^2$:

In [94]:
Product(Number(3),Power(Variable("x"),Number(2)))

In [95]:
class Sum():
    def __init__(self, *exps): #<1>
        self.exps = exps

class Function(): #<2>
    def __init__(self,name):
        self.name = name

class Apply(): #<3>
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

This represents $\cos(x^3 + -5)$:

In [96]:
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")), Number(-5)))

### Ex10.2
前の練習問題の式をPythonのコードに変換せよ。これをPythonの関数として書き、また、要素とコンビネータを用いたデータ構造にせよ。

In [97]:
from math import log

def ln(y,z):
    return log(y ** z)

ln_expression = Apply(
                    Function("ln"),
                    Power(
                        Variable("y"),
                        Variable("z")
                    ))

### Ex10.4
ある式を別の式で割ったものを表す Quotient コンビネータを実装せよ。次の式はどのように表されるか？
(a + b) / 2

In [98]:
class Quotient():
    def __init__(self, numerator, denominator):
        self.numerator = numerator
        self.denominator = denominator

# (a + b) / 2
Quotient(Sum(Variable("a"), Variable("b")), Number(2))

### Ex10.5
ある式から別の式を引いたものを表す Difference コンビネータを実装せよ。
b^2 - 4ac はどのように表せるか？

In [99]:
class Difference():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

# b^2 - 4ac
Difference(Power(Variable("b"), Number(2)), Product(Product(Number(4), Variable("a")), Variable("c")))

### Ex10.6
式の符号反転を表す Negative コンビネータを実装せよ。例えば、x^2 + y の反転は -(x^2 + y)となる。
この新しいコンビネータを使って、後者の式をコードで表せ。

In [100]:
class Negative():
    def __init__(self, exp):
        self.exp = exp

Negative(Sum(Power(Variable("x"), Number(2)), Variable("y")))

### Ex10.7
平方根を表す sqrt という関数を追加し、それを使って次の式をコード化せよ。

In [101]:
from math import sqrt as math_sqrt

def sqrt(x):
    return math_sqrt(x)

Quotient(
    Sum(
        Negative(Variable("b")),
        Apply(
                Function("sqrt"),
                Difference(
                    Power(Variable("b"), Number(2)),
                    Product(Product(Number(4), Variable("a")), Variable("c")))
        ),
    ),
    Product(Number(2), Variable("a"))
)

### Ex10.8
Expression という抽象基底クラスを作成し、すべての要素とコンビネータがそれを継承するようにする。
次に Python の算術演算 +、-、*、/ をオーバーロードし、それらが Expression オブジェクトを生成するようにする。

In [102]:
from abc import ABC

class Expression(ABC):
    def __add__(self, other):
        return Sum(self, other)

    def __sub__(self, other):
        return Difference(self, other)

    def __mul__(self, other):
        return Product(self, other)

    def __rmul__(self, other):
        return Product(other, self)

    def __truediv__(self, other):
        return Quotient(self, other)

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

class Number(Expression):
    def __init__(self,number):
        self.number = number

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol


class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps

class Function():
    def __init__(self,name):
        self.name = name

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

class Quotient(Expression):
    def __init__(self, numerator, denominator):
        self.numerator = numerator
        self.denominator = denominator

class Difference(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

class Negative(Expression):
    def __init__(self, exp):
        self.exp = exp

# 2 Putting a symbolic expression to work

## 2.1 Finding all the variables in an expression

In [103]:
def distinct_variables(exp):
    if isinstance(exp, Variable):
        return set(exp.symbol)
    elif isinstance(exp, Number):
        return set()
    elif isinstance(exp, Sum):
        return set().union(*[distinct_variables(exp) for exp in exp.exps])
    elif isinstance(exp, Product):
        return distinct_variables(exp.exp1).union(distinct_variables(exp.exp2))
    elif isinstance(exp, Power):
        return distinct_variables(exp.base).union(distinct_variables(exp.exponent))
    elif isinstance(exp, Apply):
        return distinct_variables(exp.argument)
    else:
        raise TypeError("Not a valid expression.")

In [104]:
distinct_variables(Variable("z"))

{'z'}

In [105]:
distinct_variables(Number(3))

set()

In [106]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

distinct_variables(f_expression)

{'x'}

In [107]:
distinct_variables(Sum(Variable("x"), Variable("y")))

{'x', 'y'}

## 2.2 Evaluating an expression

In [108]:
from abc import ABC, abstractmethod

class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass

Note: we are redefining these classes now.

In [109]:
class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        try:
            return bindings[self.symbol]
        except:
            raise KeyError("Variable '{}' is not bound.".format(self.symbol))

class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)

In [110]:
Product(Variable("x"), Variable("y")).evaluate(x=2,y=5)

10

In [111]:
import math
from math import sin, cos, log

_function_bindings = {
    "sin": math.sin,
    "cos": math.cos,
    "ln": math.log
}

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))

From the text: *... Similarly, we can add an “evaluate” method to the Sum, Power, Difference, or Quotient combinators....*

In [112]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)

class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)

class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)

Redefine `f_expression` in light of the new class definitions

In [113]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [114]:
f_expression.evaluate(x=5)

-76.71394197305108

In [115]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

f(5)

-76.71394197305108

## 2.3 Expanding an expression

In [116]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass
    @abstractmethod
    def expand(self):
        pass

    # Printing expressions legibly in REPL (See first mini project in 2.4)
    @abstractmethod
    def display(self):
        pass
    def __repr__(self):
        return self.display()

In [117]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    def expand(self):
        return Sum(*[exp.expand() for exp in self.exps])
    def display(self):
        return "Sum({})".format(",".join([e.display() for e in self.exps]))

class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1 = self.exp1.expand()
        expanded2 = self.exp2.expand()
        if isinstance(expanded1, Sum):
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps])
        elif isinstance(expanded2, Sum):
            return Sum(*[Product(expanded1,e) for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2)
    def display(self):
        return "Product({},{})".format(self.exp1.display(),self.exp2.display())

class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Difference({},{})".format(self.exp1.display(), self.exp2.display())

class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Quotient({},{})".format(self.numerator.display(),self.denominator.display())

class Negative(Expression):
    def __init__(self,exp):
        self.exp = exp
    def evaluate(self, **bindings):
        return - self.exp.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Negative({})".format(self.exp.display())

class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number
    def expand(self):
        return self
    def display(self):
        return "Number({})".format(self.number)

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Power({},{})".format(self.base.display(),self.exponent.display())

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        return bindings[self.symbol]
    def expand(self):
        return self
    def display(self):
        return "Variable(\"{}\")".format(self.symbol)

class Function():
    def __init__(self,name,make_latex=None):
        self.name = name
        self.make_latex = make_latex
    def latex(self,arg_latex):
        if self.make_latex:
            return self.make_latex(arg_latex)
        else:
            return " \\operatorname{{ {} }} \\left( {} \\right)".format(self.name, arg_latex)

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))
    def expand(self):
        return Apply(self.function, self.argument.expand())
    def display(self):
        return "Apply(Function(\"{}\"),{})".format(self.function.name, self.argument.display())

In [118]:
Y = Variable('y')
Z = Variable('z')
A = Variable('a')
B = Variable('b')
Product(Sum(A,B),Sum(Y,Z))

Product(Sum(Variable("a"),Variable("b")),Sum(Variable("y"),Variable("z")))

In [119]:
Product(Sum(A,B),Sum(Y,Z)).expand()

Sum(Sum(Product(Variable("a"),Variable("y")),Product(Variable("a"),Variable("z"))),Sum(Product(Variable("b"),Variable("y")),Product(Variable("b"),Variable("z"))))

In [120]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [121]:
f_expression.expand()

Sum(Product(Product(Number(3),Power(Variable("x"),Number(2))),Apply(Function("sin"),Variable("x"))),Product(Variable("x"),Apply(Function("sin"),Variable("x"))))

# 3 Finding the derivative of a function

For the rest of the notebook, I'll use the complete implementations from `expressions.py` so I don't have to re-implement every time.

In [122]:
from expressions import *

In [123]:
Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

## 3.1 Derivatives of powers

## 3.2 Derivatives of transformed functions

## 3.3 Derivatives of some special functions

## 3.4 Derivatives of products and compositions


## 3.5 Exercises

### Ex10.9
与えられた式に指定された変数が含まれているかをチェックする関数 contains(expression, variable) を書け。

In [124]:
def contains(expression, variable):
    # 式の木構造たどっていく必要がある。
    # Expression の型に応じて分岐が必要だ。
    if isinstance(expression, Sum):
        contains_result = False
        for exp in expression.exps:
            if contains(exp, variable):
                contains_result = True
                break
        return contains_result
    elif isinstance(expression, Product):
        return contains(expression.exp1, variable) or contains(expression.exp2, variable)
    elif isinstance(expression, Difference):
        return contains(expression.exp1, variable) or contains(expression.exp2, variable)
    elif isinstance(expression, Quotient):
        return contains(expression.numerator, variable) or contains(expression.denominator, variable)
    elif isinstance(expression, Negative):
        return contains(expression.exp, variable)
    elif isinstance(expression, Number):
        return False
    elif isinstance(expression, Power):
        return contains(expression.base, variable) or contains(expression.exponent, variable)
    elif isinstance(expression, Variable):
        if expression.symbol == variable.symbol:
            return True
        else:
            return False
    elif isinstance(expression, Apply):
        return contains(expression.argument, variable)
    else:
        return False

f_expression = Product(
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

print("f_expression contains x:{}".format(contains(f_expression, Variable("x"))))
print("f_expression contains y:{}".format(contains(f_expression, Variable("y"))))

f_expression contains x:True
f_expression contains y:False


### Ex10.10
式を引数として取り、式の中に現れる関数(sinやlnなど)を返す distinct_functions 関数を書け。

In [125]:
def distinct_functions(expression):
    if isinstance(expression, Sum):
        return set().union(*[distinct_functions(exp) for exp in expression.exps])
    elif isinstance(expression, Product):
        return distinct_functions(expression.exp1).union(distinct_functions(expression.exp2))
    elif isinstance(expression, Difference):
        return distinct_functions(expression.exp1).union(distinct_functions(expression.exp2))
    elif isinstance(expression, Quotient):
        return distinct_functions(expression.numerator).union(distinct_functions(expression.denominator))
    elif isinstance(expression, Negative):
        return distinct_functions(expression.exp)
    elif isinstance(expression, Number):
        return set()
    elif isinstance(expression, Power):
        return distinct_functions(expression.base).union(distinct_functions(expression.exponent))
    elif isinstance(expression, Variable):
        return set()
    elif isinstance(expression, Apply):
        s = { expression.function.name }
        return s.union(distinct_functions(expression.argument))
    else:
        return set()

expression1 = Product(
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

distinct_functions(expression1)

{'sin'}

### Ex10.11
式を取り、Sum が含まれている場合は True を返し、そうでない場合は False を返す関数 contains_sum を書け。

In [126]:
def contains_sum(expression):
    if isinstance(expression, Sum):
        return True
    elif isinstance(expression, Product):
        return contains_sum(expression.exp1) or contains_sum(expression.exp2)
    elif isinstance(expression, Difference):
        return contains_sum(expression.exp1) or contains_sum(expression.exp2)
    elif isinstance(expression, Quotient):
        return contains_sum(expression.numerator) or contains_sum(expression.denominator)
    elif isinstance(expression, Negative):
        return contains_sum(expression.exp)
    elif isinstance(expression, Number):
        return False
    elif isinstance(expression, Power):
        return contains_sum(expression.base) or contains_sum(expression.exponent)
    elif isinstance(expression, Variable):
        return False
    elif isinstance(expression, Apply):
        return contains_sum(expression.argument)
    else:
        return False


expression1 = Product(
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

expression2 = Apply(
                    Function("sin"),
                    Variable("x"))

expression3 = Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2)))

print("expression1 contains_sum:{}".format(contains_sum(expression1)))
print("expression2 contains_sum:{}".format(contains_sum(expression2)))
print("expression3 contains_sum:{}".format(contains_sum(expression3)))


expression1 contains_sum:True
expression2 contains_sum:False
expression3 contains_sum:False


# 4 Taking derivatives automatically

## 4.1 Implementing a derivative method for expressions

In [127]:
Sum(Variable("x"),Variable("c"),Number(1)).derivative(Variable("x"))

Sum(Number(1),Number(0),Number(0))

## 4.2 Implementing the product rule and chain rule

In [128]:
Product(Variable("c"),Variable("x")).derivative(Variable("x"))

Product(Variable("c"),Number(1))

In [129]:
Apply(Function("sin"),Power(Variable("x"),Number(2))).derivative(x)

Product(Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1)))),Apply(Function("cos"),Power(Variable("x"),Number(2))))

## 4.3 Implementing the power rule

In [130]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [131]:
f_expression.derivative(x)

Sum(Product(Sum(Product(Number(3),Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1))))),Number(1)),Apply(Function("sin"),Variable("x"))),Product(Sum(Product(Number(3),Power(Variable("x"),Number(2))),Variable("x")),Product(Number(1),Apply(Function("cos"),Variable("x")))))

## 4.4 Exercises

# 5 Integrating functions symbolically 

## 5.1 Integrals as antiderivatives


## 5.2 Introducing the SymPy library

In [132]:
from sympy import *
from sympy.core.core import *
Mul(Symbol('y'),Add(3,Symbol('x')))

y*(x + 3)

In [133]:
y = Symbol('y')

In [134]:
x = Symbol('x')

In [135]:
y*(3+x)

y*(x + 3)

In [136]:
y*(3+x).subs(x,1)

4*y

In [137]:
(x**2).diff(x)

2*x

In [138]:
(3*x**2).integrate(x)

x**3

### Ex10.22
f(x) = 0 の積分を求め、SymPyで確認せよ。

In [141]:
Integer(0).integrate(x)

0

### Ex10.23
xcos(x)の積分を求めよ。

In [142]:
(x * cos(x)).integrate(x)

x*sin(x) + cos(x)

### Ex10.24
x^2 の積分を求めよ。SymPyを使って答えを確認せよ。

In [143]:
(x**2).integrate(x)

x**3/3